In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch import Tensor, optim, cuda
from torch.autograd import Variable
from torch.nn import functional as F
from torch.utils.data import Dataset
import torch.nn as nn
import csv
import os
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch import Tensor, optim, cuda
from torch.autograd import Variable
from torch.nn import functional as F
from torch.utils.data import Dataset
import torch.nn as nn
import csv
import os

from data.data import DeClareDataset

# Load Datasets

In [2]:
SNOPES_LOC = "./Datasets/Snopes/snopes.tsv"
#consists of rumors analyzed on the Snopes website along with their credibility labels (true or false), 
#sets of reporting articles, and their respective web sources

POLITIFACT_LOC = "./Datasets/PolitiFact/politifact.tsv"

glove_data_file = "./Glove/glove.6B.100d.txt"

In [3]:
snopes = DeClareDataset(SNOPES_LOC, glove_data_file)

Successfully read news data from ./Datasets/Snopes/snopes.tsv
Number of articles = 29242
Number of claims = 4341
Using pre-built vocabulary


## Preliminary Analysis

In [4]:
snopes.news_df.head()

,Credibility,Claim_Source,Claim,Article,Article_Source
0,true,politics_christmas_bestbuy,best buy chain eschewing use word christmas 20...,there are several holidays throughout that tim...,www.godlikeproductions.com
1,true,politics_christmas_bestbuy,best buy chain eschewing use word christmas 20...,defenses against same photographs show images ...,www.sjpba.net
2,true,politics_christmas_bestbuy,best buy chain eschewing use word christmas 20...,the best that life could think out we extended...,www.englisher.net
3,true,politics_christmas_bestbuy,best buy chain eschewing use word christmas 20...,this entry november 19 2006 published 9 years ...,rss2.com
4,true,politics_christmas_bestbuy,best buy chain eschewing use word christmas 20...,place he will not do either said snape the ord...,www.englisher.net


In [31]:
class DeClareModel(nn.Module):
    def __init__(self, glove_embeddings, claim_source_vocab_size, article_source_vocab_size):
        super(DeClareModel, self).__init__()
        
        self.word_embeddings = nn.Embedding.from_pretrained(torch.from_numpy(glove_embeddings), freeze=False)
        self.claim_source_embeddings = nn.Embedding(claim_source_vocab_size, claim_source_vocab_size)
        self.article_source_embeddings = nn.Embedding(article_source_vocab_size, article_source_vocab_size)
        
    def forward(self, claim, article, claim_source, article_source):
        claim_mean_embedding = torch.mean(self.word_embeddings(claim), 0)
        article_embeddings = self.word_embeddings(article)
        
        demo_output = torch.cat((claim_mean_embedding.view(1,-1), article_embeddings), 0)
        return demo_output

In [32]:
declare = DeClareModel(snopes.initial_embeddings, snopes.claim_source_vocab_size, snopes.article_source_vocab_size)

In [33]:
for data_sample in snopes:
    print(declare(data_sample[0], data_sample[1], data_sample[2], data_sample[3]))
    break

tensor([[ 4.1210e-03,  1.0612e-01, -1.7659e-01,  ..., -1.0494e-03,
          6.3730e-01,  2.3561e-01],
        [-8.4215e-02,  6.9597e-01,  2.8383e-01,  ..., -5.4000e-01,
          5.8031e-01,  4.6698e-01],
        [-5.1533e-01,  8.3186e-01,  2.2457e-01,  ..., -1.2024e+00,
          1.1304e+00,  3.4790e-01],
        ...,
        [-4.6851e-01,  2.6150e-01, -3.2521e-01,  ..., -5.1552e-01,
          7.6918e-01, -1.3301e-01],
        [-2.8111e-01, -6.0057e-01, -8.3444e-02,  ..., -3.1976e-01,
         -3.7547e-01,  4.3759e-01],
        [-5.9779e-02,  2.5019e-01, -2.0438e-01,  ...,  4.0870e-01,
          7.4594e-02, -1.6164e-01]], dtype=torch.float64,
       grad_fn=<CatBackward>)
